Need to check list of words for repeats, need unique list
Check importance of double letters (is prioritizing more characters more valuable than repeats of higher "value" letters)
Increase char prop values by 1000 to make values more readable

In [ ]:
# load packages

import pandas as pd
import numpy as np
np.random.seed(59)
import seaborn as sns
import math
import string

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics 

import urllib.request

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
# read in list of words to be used as database of answers

url = "https://www-cs-faculty.stanford.edu/~knuth/sgb-words.txt"
file = urllib.request.urlopen(url)
db = []

for line in file:
    decoded_line = line.decode("utf-8")
    db.append(decoded_line)
    
db

In [ ]:
# cleaning up list, converting to upper case to match wordle

db_temp = []

for x in db:
    cut_string = x[:5]
    db_temp.append(cut_string.upper())
    
db_temp 
db = db_temp.copy()
db_temp.clear()

db

In [ ]:
# checking length of list

len(db)

In [ ]:
# function for splitting words

def split(word):
    return [char for char in word]

In [ ]:
# function for determing distribution of letters, based upon original data set. Also determined letter weighting

char_count = {}
char_totals = {}
char_st = {}
char_temp = []

def char_totals_func(word_list):

    char_count.clear()
    
    for word in word_list:
        char_temp = split(word)

        for char in char_temp:
            if (char in char_count):
                char_count[char] += 1
            else:
                char_count[char] = 1
                
    char_totals.clear()
    
    sorted_chars = dict(sorted(char_count.items(), key = lambda x: x[0].lower()))
    
    for k,v in sorted_chars.items():
        char_totals[k] = v
    
    print(char_totals)

    # confirming values match total words times 5 letters
    print('Expected Number = ' + str(len(word_list)*5))
    print('Sum of letter count = ' + str(sum(char_totals.values())))

    plt.bar(list(char_totals.keys()), char_totals.values(), color = 'b')
    plt.show()
    
    char_st.clear()

    for k,v in char_totals.items():
        char_st[k] = v/sum(char_totals.values())

    char_max = (max(char_st.values()))

    for k,v in char_st.items():
        char_st[k] = (v/char_max)*1

In [ ]:
# running counter on original database           
char_totals_func(db)

In [ ]:
# function for sorting data set to only words in five unique values

db_uniques = []

def db_uniques_func(word_list):

    db_uniques.clear()
    
    for word in word_list:
        char_temp = split(word)

        if (len(set(char_temp)) == len(word)): db_uniques.append(word)

    print('Filtered list length: ' + str(len(db_uniques)))
    print(str(len(word_list)-len(db_uniques)) + ' words have been removed')

In [ ]:
# sorting for uniques on original data set
db_uniques_func(db)

In [ ]:
# function for weighting words in list. Gives highest strength word in list
# db stores words

word_st = {}
char_st_temp = {}

def word_st_func(word_list):
    
    word_st.clear()
    
    for word in word_list:
        char_temp = split(word)

        char_st_temp = {}
        word_st_temp = 1


        for char in char_temp:
            if (char in char_st_temp):
                char_st_temp[char] = char_st_temp[char] * char_st[char]

            else:
                char_st_temp[char] = char_st[char]

        for char in char_st_temp:
            word_st_temp = word_st_temp * char_st_temp[char]


        word_st[word] = word_st_temp

In [ ]:
# weighting each word in the filtered, unique data set

word_st_func(db_uniques)
print("Highest strength words: " + str(sorted(word_st, key=word_st.get, reverse=True)[:5]))

In [ ]:
# filtering based upon responses

word_played_r1 = 'AROSE'

In [ ]:
# key: 0 = gray, 1 = yellow, 2 = green

word_played_r1_result = [0,2,0,0,0]

#creating list of possible letters for each slot based upon database

#creating dictionary that reflects all possible characters for each slot
#char_loc_ref = dicionary containing all possible letters for each slot
#key = char_temp.index(char)
#value = char

for word in db:
    char_temp = []
    
    char_temp = split(word)
    
    for char in char_temp:
        if char_temp.index(char) not in char_loc_ref:
            char_loc_ref[char_temp.index(char)] = char 
        elif isinstance (char_loc_ref[char_temp.index(char)], list):
            if char not in char_loc_ref[char_temp.index(char)]:
                char_loc_ref[char_temp.index(char)].append(char)
        else:
            char_loc_ref[char_temp.index(char)] = [char_loc_ref[char_temp.index(char)], char]
            
for slot in char_loc_ref:
    char_loc_ref[slot] = sorted(char_loc_ref[slot])
    
char_loc_ref

In [ ]:
# creating list of all possible letters per slot

char_loc_ref = {}

for i in range(5):

    char_loc_ref[i] = "A"
    char_loc_ref[i] = [char_loc_ref[i], "B"]

    for char in string.ascii_uppercase[2:]:
        char_loc_ref[i].append(char)


In [ ]:
# function for filtering possible characters for each slot based upon result
   
char_include = []
    
def char_loc_ref_filt_func(word_played_rX, word_played_rX_result):
    
    char_temp = split(word_played_rX)

    for i in range(len(word_played_rX_result)):
        if word_played_rX_result[i] == 0:
            for slot in char_loc_ref:
                if char_temp[i] in char_loc_ref[slot] and len(char_loc_ref[slot]) != 1: char_loc_ref[slot].remove(char_temp[i])

        if word_played_rX_result[i] == 1:
            # Kept to account for answer not being in data set, leading to error
            if char_temp[i] in char_loc_ref[i]: char_loc_ref[i].remove(char_temp[i])

            if char_temp[i] not in char_include: char_include.append(char_temp[i])

        if word_played_rX_result[i] == 2:
                char_loc_ref[i].clear()
                char_loc_ref[i].append(char_temp[i])

    print(char_loc_ref)

In [ ]:
# running filtering character reference list based on first round
char_loc_ref_filt_func(word_played_r1, word_played_r1_result)

In [ ]:
# function for filtering data set based upon possible character values

def db_filtered_func(db_to_filt):
    
    db_filtered_temp = db_to_filt.copy()

    for word in db_to_filt:
        char_temp = split(word)

        for i in range(len(char_temp)):
            if char_temp[i] not in char_loc_ref[i]:
                db_filtered_temp.remove(word)            
                break   

    db_filtered_temp_output = db_filtered_temp.copy()
    
    for word in db_filtered_temp_output:
        char_temp = split(word)

        for char in char_include:
            if char not in char_temp:
                db_filtered_temp.remove(word)
                break

    global db_filtered
    db_filtered = db_filtered_temp.copy()

    print(db_filtered)
    print("Number of Possible Answers: " + str(len(db_filtered)))

In [ ]:
# filtered data set based upon resulted of first round
db_filtered_func(db)

In [ ]:
# determing distribution of letters, based upon filtered data set, determining new letter weighting
# filtering for words with five unique letters on filtered data set
# weighting words in new filtered list. Must include "yellow" letters

char_totals_func(db_filtered)
db_uniques_func(db_filtered)
word_st_func(db_uniques)
print("Highest strength words: " + str(sorted(word_st, key=word_st.get, reverse=True)[:5]))

In [ ]:
# word played for round 2

word_played_r2 = 'PRINT'

In [ ]:
# key: 0 = gray, 1 = yellow, 2 = green

word_played_r2_result = [0,2,0,2,0]

In [ ]:
# running filtering character reference list based on second round
char_loc_ref_filt_func(word_played_r2, word_played_r2_result)

In [ ]:
# filtered data set based upon results of second round
db_filtered_func(db_filtered)

In [ ]:
# determing distribution of letters, based upon filtered data set, determining new letter weighting
# filtering for words with five unique letters on filtered data set
# weighting words in new filtered list. Must include "yellow" letters

char_totals_func(db_filtered)
db_uniques_func(db_filtered)
word_st_func(db_uniques)
print("Highest strength words: " + str(sorted(word_st, key=word_st.get, reverse=True)[:5]))

In [ ]:
# word played for round 3

word_played_r3 = 'DRUNK'

In [ ]:
# key: 0 = gray, 1 = yellow, 2 = green

word_played_r3_result = [0,2,2,2,0]

In [ ]:
# running filtering character reference list based on second round
char_loc_ref_filt_func(word_played_r3, word_played_r3_result)

In [ ]:
# filtered data set based upon results of third round
db_filtered_func(db_filtered)

In [ ]:
# determing distribution of letters, based upon filtered data set, determining new letter weighting
# words with multiple letters allowed for fourth round and on
# weighting words in new filtered list. Must include "yellow" letters

char_totals_func(db_filtered)
word_st_func(db_filtered)
print("Highest strength words: " + str(sorted(word_st, key=word_st.get, reverse=True)[:5]))

Words with multiple usage of same letter allowed for fourth round and on

In [ ]:
# weighting words in new filtered list. Must include "yellow" letters

word_st_func(db_filtered)
print("Highest strength words: " + str(sorted(word_st, key=word_st.get, reverse=True)[:5]))

In [ ]:
# word played for round 4

word_played_r4 = 'BRUNG'

In [ ]:
# key: 0 = gray, 1 = yellow, 2 = green

word_played_r4_result = [2,2,2,2,2]

In [ ]:
# running filtering character reference list based on fourth round
char_loc_ref_filt_func(word_played_r4, word_played_r4_result)

In [ ]:
# filtered data set based upon results of third round
db_filtered_func(db_filtered)